<a href="https://colab.research.google.com/github/rishabh-2005/IITISoC-ML-05/blob/main/object_inpainting/auto_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics opencv-python-headless diffusers torch torchvision ffmpeg-python
!git clone https://github.com/sczhou/ProPainter.git
%cd ProPainter

!pip install -r requirements.txt

import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


In [ ]:
import subprocess

In [ ]:
def has_audio_stream(video_path):
    """Check if a video has an audio stream using ffprobe."""
    result = subprocess.run(
        ["ffprobe", "-i", video_path, "-show_streams", "-select_streams", "a", "-loglevel", "error"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    return result.stdout != b""

In [ ]:
def run_pipeline(input_path, output_path="/content/outputs/final.mp4"):
    from ultralytics import YOLO
    import cv2
    import os
    import numpy as np
    import shutil
    import subprocess

    def generate_mask(frame, results):
        mask = np.zeros_like(frame[:, :, 0], dtype=np.uint8)
        for result in results:
            if result.masks is not None:
                for seg_mask in result.masks.data:
                    seg_mask_np = seg_mask.cpu().numpy().astype(np.uint8)
                    seg_mask_resized = cv2.resize(seg_mask_np, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)
                    mask = cv2.bitwise_or(mask, seg_mask_resized * 255)
                    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
                    mask = cv2.dilate(mask, kernel, iterations=1)
                    mask = cv2.GaussianBlur(mask, (5,5), 0)
        return mask

    # Load YOLO model
    model = YOLO("yolov8x-seg.pt")

    # Prepare folders
    mask_folder = "/content/yolo_seg_masks"
    os.makedirs(mask_folder, exist_ok=True)

    # Read video and create masks
    cap = cv2.VideoCapture(input_path)
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame)
        mask = generate_mask(frame, results)
        mask_path = os.path.join(mask_folder, f"frame_{frame_idx:04d}.png")
        cv2.imwrite(mask_path, mask)
        frame_idx += 1
    cap.release()

    import os

    if not os.path.exists("/content/ProPainter"):
      !git clone https://github.com/sczhou/ProPainter.git
    %cd /content/ProPainter
    !pip install -r requirements.txt


    # Run ProPainter
    !python /content/ProPainter/inference_propainter.py \
        --video {input_path} \
        --mask {mask_folder} \
        --output /content/outputs \
        --width 640 --height 360 \
        --subvideo_length 40

    # Output path is assumed
    %cd /content


    # Output path from ProPainter
    inpainted_path = "/content/outputs/input/inpaint_out.mp4"
    audio_path = "/content/original_audio.aac"
    output_path = "/content/outputs/final.mp4"


    # Step 1: Extract audio from original video
    if has_audio_stream(input_path):
        print("🔊 Extracting and adding audio...")
        subprocess.run(["ffmpeg", "-y", "-i", input_path, "-vn", "-acodec", "copy", audio_path], check=True)
        subprocess.run([
            "ffmpeg", "-y", "-i", inpainted_path, "-i", audio_path,
            "-c:v", "copy", "-c:a", "aac", "-strict", "experimental", output_path
        ], check=True)
    else:
        print("⚠️ No audio stream found. Skipping audio operations.")
        shutil.copy(inpainted_path, output_path)

    shutil.rmtree(mask_folder, ignore_errors=True)
    return output_path

In [ ]:
!pip install fastapi uvicorn python-multipart pyngrok

from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse
from pyngrok import ngrok
import shutil
import requests
import os

app = FastAPI()

from fastapi import HTTPException

@app.post("/process/")
async def process_url(video_url: str = Form(...)):
    try:
        print("📥 Received URL:", video_url)

        input_path = "/content/input.mp4"
        output_path = "/content/outputs/final.mp4"

        print("🌐 Downloading video...")
        response = requests.get(video_url, stream=True)
        if response.status_code != 200:
            raise HTTPException(status_code=400, detail="Video download failed.")

        with open(input_path, "wb") as f:
            f.write(response.content)
        print("✅ Video saved.")

        print("🎬 Running pipeline...")
        run_pipeline(input_path)
        print("✅ Pipeline completed.")

        if not os.path.exists(output_path):
            raise HTTPException(status_code=500, detail="Output video not found.")

        print("📤 Sending back file:", output_path)
        return FileResponse(output_path, media_type="video/mp4", filename="inpainted_video.mp4")

    except Exception as e:
        print("❌ ERROR:", str(e))
        raise HTTPException(status_code=500, detail=f"Processing failed: {str(e)}")



In [ ]:
!ngrok config add-authtoken "YOUR_AUTH_TOKEN"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import threading
import uvicorn
from pyngrok import ngrok

# Start FastAPI server in background
def run():
    uvicorn.run(app, host="0.0.0.0", port=3000)

thread = threading.Thread(target=run)
thread.start()

#  Start ngrok tunnel
public_url = ngrok.connect(8000)
print("🚀 Public URL:", public_url)


In [ ]:
# !curl -X POST https://67a9-34-87-1-86.ngrok-free.app/process/ \
#   -F "video_url=https://res.cloudinary.com/dahwwh212/video/upload/v1750508813/user1/dl1brgjfgxcvexslxdfu.mp4" \
#   --output inpainted_with_audio.mp4

In [ ]:
# !curl https://4fb0-34-125-14-251.ngrok-free.app/process/
